In [ ]:
from ultralytics import YOLO
import cv2

# Load the YOLOv8 model (replace with yolov8x.pt for the large model)
model = YOLO(r"E:\Projects\Code-Blue\yolov8n.pt")

# Read the image
image_path = r'e:\Projects\Code-Blue\Code-Blue\image.png'  # Replace with your image's path
image = cv2.imread(image_path)

# Perform object detection
results = model(image)

# Visualize the results
annotated_image = results[0].plot()  # Annotate the image with detections

# Display the annotated image
cv2.imshow("Detected Objects", annotated_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\Shiva\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.

0: 640x640 1 frisbee, 5.3ms
Speed: 5.9ms preprocess, 5.3ms inference, 138.5ms postprocess per image at shape (1, 3, 640, 640)
